In [ ]:
!pip install -r requirements.txt

  Using cached CFEDemands-0.4.1-py2.py3-none-any.whl (39 kB)
  Using cached ConsumerDemands-0.3.dev0-py2.py3-none-any.whl (12 kB)
  Using cached oauth2client-4.1.3-py2.py3-none-any.whl (98 kB)
  Using cached eep153_tools-0.11-py2.py3-none-any.whl (4.4 kB)
Processing /home/jovyan/.cache/pip/wheels/20/7e/30/7d702acd6a1e89911301cd9dbf9cb9870ca80c0e64bc2cde23/gnupg-2.3.1-py3-none-any.whl


### From Sheet to DataFrame



We begin by defining a dictionary that contains the spreadsheet key.

In [ ]:
nigeria_data = '17L5cDhXRLNAckP3JvBLTLSYIguFqP2ebMvQLH96c0n4'
nigeria_production = '1kG_fVBmj9EEF9LOwxN30HBxkQENOoWeQjVPYzMJe3b4-8DA'
nigeria_consumption = '1Gzu0g2Tjp0heKYk-r5l7gVJk93k2_mHgWOtZMVkqSGI'

With the spreadsheet defined, grab it and define a couple of
dataframes.



In [ ]:
import pandas as pd
import numpy as np
import sys
from eep153_tools.sheets import read_sheets

expend = read_sheets(nigeria_data,sheet='Expenditures')
expend.columns.name = 'i'
                 
# Change 'ICRISAT' to key of your own sheet in Sheets, above
hh_char = read_sheets(nigeria_data,sheet="HH Characteristics")
hh_char.columns.name = 'k'

# Assume a single market: (Comment this out to make each village its own market)
hh_char['m'] = 1
expend['m'] = 1

# x may have duplicate columns
expend = expend.groupby('i',axis=1).sum()
expend = expend.apply(lambda x: pd.to_numeric(x,errors='coerce'))
expend = expend.replace(0,np.nan)

# Take logs of expenditures; call this y
log_expend = np.log(expend.set_index(['j','t','m']))
           
hh_char.set_index(['j','t','m'],inplace=True)

Sort the new Data Frame in order to group by year.

In [ ]:
expend = expend.set_index(['t','j','m']).sort_index()
expend = expend.replace(0.0,np.nan) # Replace zeroes with np.nan
expend

# People per Household, Total Expenditures, and Expenditures per Capita

In [ ]:
people = hh_char.sum(axis=1)
num_people = pd.DataFrame(people)
num_people = num_people.rename(columns={0:'People per HH'})
num_people = num_people.reset_index().set_index(['t','j','m']).sort_index()
num_people

In [ ]:
total_expend = expend.iloc[:, 0:124].sum(axis=1)
total = pd.DataFrame(total_expend)
total = total.rename(columns={0:'Total Expenditures'})
total

In [ ]:
expend['Total Expenditures'] = total['Total Expenditures']
expend['People per HH'] = num_people['People per HH']
expend['Expenditures per capita'] = expend['Total Expenditures'] / expend['People per HH']
expend

In [ ]:
expend.info()

# Putting into Quartiles

In [ ]:
def one_year(df, year):
    new_df = df.loc[[year]]
    return new_df

def quartiles_by_te(df, year, quartile):
    # Selecting out one year, sorting by TE, then filtering out the HHs that spent nothing
    one_year_df = one_year(df, year)
    one_year_df = one_year_df.reset_index().sort_values('Total Expenditures', axis=0).replace(0,np.nan)
    one_year_df = one_year_df.dropna(axis=0, how='any', subset=['Total Expenditures'])
    
    # Number of rows for each quartile
    total_rows = len(one_year_df)
    rows_per_qtr = round(total_rows / 4)
    
    # Selecting the necessary rows for each quartile
    if quartile == 1:
        return one_year_df.iloc[0:rows_per_qtr-1]
    else:
        first_row = (quartile-1) * rows_per_qtr
        last_row = (quartile * rows_per_qtr) - 1
        return one_year_df.iloc[first_row:last_row]
    
def quartiles_by_epc(df, year, quartile):
    # Selecting out one year, sorting by EPC, then filtering out the HHs that spent nothing
    one_year_df = one_year(df, year)
    one_year_df = one_year_df.reset_index().sort_values('Expenditures per capita', axis=0).replace(0,np.nan)
    one_year_df = one_year_df.dropna(axis=0, how='any', subset=['Expenditures per capita'])
    
    # Number of rows for each quartile
    total_rows = len(one_year_df)
    rows_per_qtr = round(total_rows / 4)
    
    # Selecting the necessary rows for each quartile
    if quartile == 1:
        return one_year_df.iloc[0:rows_per_qtr-1]
    else:
        first_row = (quartile-1) * rows_per_qtr
        last_row = (quartile * rows_per_qtr) - 1
        return one_year_df.iloc[first_row:last_row]

In [ ]:
one_year(expend, 2012)

In [ ]:
first_qtr_by_te_2010 = quartiles_by_te(expend, 2010, 1)
first_qtr_by_te_2010
#te is total expenditure

In [ ]:
Q1_10 = quartiles_by_epc(expend, 2010, 1)
Q1_12 = quartiles_by_epc(expend, 2012, 1)
Q1_15 = quartiles_by_epc(expend, 2015, 1)
Q1_18 = quartiles_by_epc(expend, 2018, 1)
Q1 = pd.concat([Q1_10, Q1_12, Q1_15, Q1_18]).reset_index().drop(columns=['index']).set_index(['t', 'j', 'm']).sort_values(['t','j'])
Q1 = Q1.drop(columns=['Total Expenditures', 'People per HH', 'Expenditures per capita']) 
Q1
#epc is expenditure per capita

In [ ]:
Q4_10 = quartiles_by_epc(expend, 2010, 4)
Q4_12 = quartiles_by_epc(expend, 2012, 4)
Q4_15 = quartiles_by_epc(expend, 2015, 4)
Q4_18 = quartiles_by_epc(expend, 2018, 4)
Q4 = pd.concat([Q4_10, Q4_12, Q4_15, Q4_18]).reset_index().drop(columns=['index']).set_index(['t', 'j', 'm']).sort_values(['t','j'])
Q4 = Q4.drop(columns=['Total Expenditures', 'People per HH', 'Expenditures per capita']) 
Q4

## Filter Household Dataframe to create one only including 1st quartile households and another including just 4th quartile households.

In [ ]:
#First Quartile
hh_char = hh_char.reorder_levels(['t','j','m'])
Q1Index = Q1.index.tolist()
Q4Index = Q4.index.tolist()
hh_charQ1 = hh_char[hh_char.index.isin(Q1Index)]
hh_charQ4 = hh_char[hh_char.index.isin(Q4Index)]
hh_charQ1

In [ ]:
#Fourth Quartile
hh_charQ4

In [ ]:
#Logged Food Expenditure Dataframe (after running np.log on values)

Q1 = Q1.replace(0,np.nan) 
Q4 = Q4.replace(0,np.nan) 

log_Q1 = np.log(Q1)
log_Q4 = np.log(Q4)

In [ ]:
log_Q1

In [ ]:
#Log Household Size and add to household dataframe for Q1 and Q4

# set index to j, t, m so that df.sum() ignore index values
hh_charQ1 = hh_charQ1.reset_index()
hh_charQ1.set_index(['j','t','m'], inplace=True)
hh_charQ4 = hh_charQ4.reset_index()
hh_charQ4.set_index(['j','t','m'], inplace=True)

# create new column of household size
hh_charQ1['Hsize'] = hh_charQ1.sum(axis=1).values
hh_charQ4['Hsize'] = hh_charQ4.sum(axis=1).values

# remove erroneous data with household_size = 0
hh_charQ1 = hh_charQ1[hh_charQ1['Hsize'] > 0]
hh_charQ4 = hh_charQ4[hh_charQ4['Hsize'] > 0]

# create new column 'log Hsize'
hh_charQ1['log Hsize'] = np.log(hh_charQ1['Hsize'])
hh_charQ4['log Hsize'] = np.log(hh_charQ4['Hsize'])

# remove Hsize column
hh_charQ1 = hh_charQ1.drop(columns=['Hsize']) 
hh_charQ4 = hh_charQ4.drop(columns=['Hsize']) 

In [ ]:
#test
hh_charQ1

## Estimation



With nothing more than this, we can estimate the demand system.  This
happens in two steps.  The first is the &ldquo;reduced form&rdquo; step:



In [ ]:
log_Q1

In [ ]:
import cfe

result1 = cfe.Result(y=log_Q1,z=hh_charQ1, min_xproducts = 10)
#result4 = cfe.Result(y=log_Q4,z=hh_charQ4)

This creates a complicated &ldquo;Result&rdquo; object, with lots of different
attributes.  Note from below that attributes $y$ and $z$ are now defined.



In [ ]:
result1

### First step of Estimation



Recall that there are two steps to estimation; the first step
involves estimating the &ldquo;reduced form&rdquo; linear regression 
$$
y_{it}^j = {a}_{it} + \delta_i'{z}^j_t + \epsilon_{it}^j.
$$

The Result class has code to estimate this in one line:



In [ ]:
result1.get_reduced_form()
#result4.get_reduced_form()

After running this we can examine the estimated coefficients $\delta$:



In [ ]:
result1.delta.to_dataframe().unstack('k')
result4.delta.to_dataframe().unstack('k')

Also the good-time constants $a_{it}$ (this captures the effects of prices)



In [ ]:
result1.a.to_dataframe().unstack('i')

### Second step of Estimation



The second step involves using Singular Value Decomposition to find
the rank one matrix that best approximates the residuals $e_{it}^j$.
This can be interpreted as
$$
    -\beta_i\log\lambda^j_t,
$$
where the $\log\lambda^j_t$ is the log of the marginal utility of
expenditures (MUE) for household $j$ at time $t$, and where $\beta_i$ are
the corresponding &ldquo;Frisch elasticities&rdquo; that tell us how much
demand changes as the MUE falls.

Estimates can also be computed as a one-liner:



In [ ]:
result1.get_beta(as_df=True)

That&rsquo;s all there is to estimation!  Note that we didn&rsquo;t estimate
demands for all goods&#x2014;lots of goods didn&rsquo;t have enough observations,
and were automatically dropped.  (This can be controlled using the
`min_proportion_items` and `min_xproducts` attributes when one
instantiates the result object.)



### Assessment of Fit



Now, let&rsquo;s see how we did, by comparing total expenditures predicted by the
model we&rsquo;ve estimated with actual total expenditures:



In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.cm as cm

xbar = np.exp(result.y).sum(['m','i']).to_dataframe('xbar').replace(0,np.nan).squeeze()
xhat = result.get_predicted_expenditures().sum(['m','i']).to_dataframe('xhat').replace(0,np.nan).squeeze()

# Make dataframe of actual & predicted
df = pd.DataFrame({'Actual':np.log(xbar),'Predicted':np.log(xhat)})

df.plot.scatter(x='Predicted',y='Actual')

# Add 45 degree line
v = plt.axis()
vmin = np.max([v[0],v[2]])
vmax = np.max([v[1],v[3]])
plt.plot([vmin,vmax],[vmin,vmax])

In [ ]:
result.to_dataset('icrisat.ds')

### Nutritional Data

In [ ]:
fdc_table = '1ed8FASRCkN9KwTWTvMzKT6UT4jWbSSZQEwZEmXCt8IQ'

fdc_codes = read_sheets(fdc_table,sheet="Sheet1")

consumption = read_sheets(nigeria_consumption,sheet='Consumption')

In [ ]:
import fooddatacentral as fdc

apikey = 'MfcTfizjo11bsqJeJCn9Tb7RdKPQxJRjJSvTKElr'

food_nutrients = {}
for f in fdc_codes['Food description'].to_list():
    fdc_id = fdc_codes[fdc_codes['Food description'] == f]['USDA FDC ID'].values[0]
    if not np.isnan(fdc_id):
        try:
            fdc_id = int(fdc_id)
            food_nutrients[f] = fdc.nutrients(apikey, fdc_id).Quantity
        except AttributeError:
            warnings.warn("Couldn't find FDC Code %s for food %s." % (f, fdc_id))

nutritional_df = pd.DataFrame(food_nutrients,dtype=float)

In [ ]:
dri_mins_sheet = '1XJRHTnxNJwrUXperIhwrwDp1HcVxPEVoQobYDmjg9Qw'

dri_mins = read_sheets(dri_mins_sheet,sheet='diet_minimums')
dri_mins = dri_mins.reset_index(drop=True).set_index('Nutrition').drop('Source', axis=1)
dri_mins